#Telecom Domain ReadOps Assignment
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
create catalog if not exists telecom_catalog_assign;
create schema if not exists telecom_catalog_assign.landing_zone ;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;


In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

### a. Volume vs DBFS 

**Volumes** are Unity Catalog-governed objects that provide a logical layer for managing non-tabular data (files) within cloud object storage with centralized governance. They are the modern, recommended approach in Databricks and support access control policies and lineage tracking through Unity Catalog.  
**DBFS/FileStore** (Databricks File System) is an older abstraction over cloud storage that allowed users to interact with data using simple paths or mounts without robust, centralized governance. The DBFS root and its mounts are deprecated, and Databricks recommends migrating to volumes or external locations under Unity Catalog.


| Feature                | Volumes (Unity Catalog)                                                                 | DBFS/FileStore (Deprecated)                                  |
|------------------------|----------------------------------------------------------------------------------------|--------------------------------------------------------------|
| Governance             | Centralized, fine-grained access control via Unity Catalog                             | Limited, workspace-level controls                            |
| Data Lineage           | Supported                                                                              | Not supported                                                |
| Recommended for Prod   | Yes                                                                                    | No                                                           |
| Access Control         | Unity Catalog policies                                                                 | ACLs, less granular                                          |
| Usage                  | Non-tabular data, files, ML models, etc.                                               | General file storage                                         |
| Migration              | Modern, recommended approach                                                           | Deprecated, migrate to Volumes or External Locations         |



###b. Why production teams prefer Volumes for regulated data


Production teams prefer Volumes for regulated data primarily due to their integration with Unity Catalog, which provides a robust framework for data governance, security, and access management. 

**Centralized Governance and Auditing:** Unity Catalog provides a single place to manage data access, permissions, and auditing across all Databricks workspaces. This is essential for meeting compliance requirements for regulated data.

**Granular Access Control:** Volumes allow administrators to define precise access controls on specific volumes or subfolders within the cloud storage location, which is critical for restricted and sensitive data.

**Simplified Compliance:** The structured, governed approach of volumes simplifies compliance reporting and ensures data is handled consistently according to organizational policies and industry regulations.

**Lifecycle Management:** Volumes have their own policies for permissions, encryption, backup, and recovery, which aids in managing the full data lifecycle in a compliant manner. 

**DBFS** lacks these centralized governance features, making it difficult to enforce consistent, auditable access controls required for sensitive or regulated production data. 

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

##2. Filesystem operations

###Loading the datasets into Volume

1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/


In [0]:
#Loading the customer data into customer.csv file

customer_csv = ''' 101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,
104,Raj,52,Mumbai,
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID '''
dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv",customer_csv,overwrite=True)

#Loading the usage data into usage.tsv file

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''
dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",usage_tsv,overwrite=True)

#Loading the tower log data into regionwise .tsv file

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''
dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower1/tower_logs_region1.csv",tower_logs_region1,overwrite=True)
dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower2/tower_logs_region2.csv",tower_logs_region1,overwrite=True)


In [0]:
dbutils.fs.help()

###Verify if data is loaded
2. Write a command to validate whether files were successfully copied

In [0]:
#validating if customer file is loaded successfully
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.head("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")

In [0]:
#validating if usage file is loaded successfully
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.head("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")


In [0]:
#validating if tower log file is loaded successfully
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower1/")
dbutils.fs.head("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower1/tower_logs_region1.csv")

dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower2/")
dbutils.fs.head("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower2/tower_logs_region2.csv")

##3. Directory Read Use Cases


###3.1. Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup

In [0]:
pth_glb_fl_df1=spark.read.csv(path=["dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower1","dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower2"],header=True,inferSchema=True,sep='|',pathGlobFilter="*.csv",recursiveFileLookup=True)
pth_glb_fl_df1.show()
display(pth_glb_fl_df1)





###3.2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

In [0]:
#Using pathGlobFilter

pth_glb_fl_df1=spark.read.csv(path=["dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower1","dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower2"],header=True,inferSchema=True,sep='|',pathGlobFilter="*.csv",recursiveFileLookup=True)
pth_glb_fl_df1.show()
display(pth_glb_fl_df1)


In [0]:
#Using list of paths in spark.read.csv([path1, path2]) 

mul_pth_df1=spark.read.csv(path:=["dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower1/","dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower2/"],header=True,inferSchema=True, sep='|',recursiveFileLookup=True)
mul_pth_df1.show()
display(mul_pth_df1)

In [0]:
#Using .option("recursiveFileLookup","true")

mul_pth_df2= spark.read.option("header",True).option("delimiter","|").option("recursiveFileLookup",True).format("csv").load(["dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/"])
mul_pth_df2.show()
display(mul_pth_df2)


###3.3 Compare the outputs and understand when each should be used.

- pathGlobFilter="*.csv" -This can be used to read files of specific(*.csv) format within the specifed path/folder
- List of paths -This can be used to read files from multiple paths/sources
- Option -This can be used for more than one options, and parameters can be passed to the option & recursiveFileLookup can be used to read files from all the sub folders in the mentioned path

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
##Read cust file with option and header=true, inferSchema=true
cust_df1=spark.read.option("header",True).option("inferSchema",True).option("delimiter",",").format("csv").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(cust_df1)
print(cust_df1.schema)

##Read cust file with options and header=true, inferSchema=true
cust_df2=spark.read.options(header=True,inferSchema=True,delimiter=",").format("csv").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(cust_df2)
print(cust_df2.schema)

#Read cust file with options.csv and header=true, inferSchema=true
cust_df3=spark.read.options(header=True,inferSchema=True,delimiter=",").csv("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(cust_df3)
print(cust_df3.schema)


In [0]:

##Read cust file with option and header=false, inferSchema=false
cust_df4=spark.read.option("header",False).option("inferSchema",False).option("delimiter",",").format("csv").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(cust_df4)
print(cust_df4.schema)

##Read cust file with options and header=false, inferSchema=false
cust_df5=spark.read.options(header=False,inferSchema=False,delimiter=",").format("csv").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(cust_df5)
print(cust_df5.schema)

#Read cust file with options.csv and header=False, inferSchema=false
cust_df6=spark.read.options(header=False,inferSchema=False,delimiter=",").csv("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(cust_df6)
print(cust_df6.schema)

In [0]:
##Read usage file with option and header=True, inferSchema=True
usage_df1=spark.read.option("header",True).option("inferSchema",True).option("delimiter","\t").format("csv").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")
display(usage_df1)
print(usage_df1.schema)

##Read usage file with options and header=True, inferSchema=True
usage_df2=spark.read.options(header=True,inferSchema=True,delimiter="\t").format("csv").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")
display(usage_df2)
print(usage_df2.schema)

##Read usage file with options.csv and header=True, inferSchema=True
usage_df3=spark.read.options(header=True,inferSchema=True,delimiter="\t").csv("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")
display(usage_df3)
print(usage_df3.schema) 

##Read usage file with option and header=False, inferSchema=False
usage_df4=spark.read.option("header",False).option("inferSchema",False).option("delimiter","\t").format("csv").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")
display(usage_df4)
print(usage_df4.schema)

##Read usage file with options and header=False, inferSchema=False
usage_df5=spark.read.options(header=False,inferSchema=False,delimiter="\t").format("csv").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")
display(usage_df5)
print(usage_df5.schema)

##Read usage file with options.csv and header=False, inferSchema=False
usage_df6=spark.read.options(header=False,inferSchema=False,delimiter="\t").csv("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")
display(usage_df6)
print(usage_df6.schema) 


Write a note on What changed when we use header or inferSchema with true/false?

- **Header = True** - When header is assigned as True its considering the first row as header
- **Header = False** - When header is assigned as False its assigning header as c0, c1, c2..etc.,
- **InferSchema = True** - When inferSchema is assigned as True it scans all rows and assigns datatype accordingly. It can be used in small datasets, but in huge volume of data full scan takes time and needs to be used carefully. Sampling ratio option can be used to avoid full scanning.
- **InferSchema = False** -When inferSchema is assigned as False all the columns are treated as strings

How schema inference handled “abc” in age?

The column age has one value as "abc", even when InferSchema is True its assigned as string datatype, to be on safer side.

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

###5.1 Apply column names using string using toDF function for customer data

In [0]:
cust_df=spark.read.csv("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv",header=True,inferSchema=True).toDF("customer_id","name","age","city","plan")
display(cust_df)
print(cust_df.schema)

###5.2 Apply column names and datatype using the schema function for usage data

In [0]:
str_struct= "customer_id integer,voice_mins integer,data_mb integer,sms_count integer"
usage_df=spark.read.schema(str_struct).csv("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",header=True,sep="\t")
display(usage_df)
print(usage_df.schema)

###5.3 Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
custom_schema=StructType([StructField("id",IntegerType(),False),StructField("fname",StringType(),True),StructField("lname",StringType(),True),StructField("event_id",IntegerType(),True),StructField("customer_id",IntegerType(),True),StructField("tower_id",StringType(),True),StructField("signal_strength",IntegerType(),True),StructField("timestamp",StringType())])
usage_df_df=spark.read.schema(custom_schema).csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/sourcedata/custs_header_1")
print(usage_df.printSchema())
usage_df.show(2)

## 6. More to come (stay motivated)....